In [1]:
import pandas as pd
import numpy as np

In [2]:
def normalize_types(df):
    for column in df.columns[df.dtypes == 'float64']:
        df[column] = df[column].astype(np.float32)

id = 1
buildings = pd.read_csv(f'./data/buildings_{id}.csv')
time_series = pd.read_csv(f'./data/time_series_{id}.csv')

normalize_types(buildings)
normalize_types(time_series)

print(buildings.shape)
print(time_series.shape)

(507, 20)
(3513816, 15)


1. Z jednego okresu
2. Jeden typ budynków
3. Jedna szerokość geograficzna
4. Zastosowany model
5. Okna czasowe predykcji

In [3]:
buildings.describe()    

,energystarscore,numberoffloors,occupants,sqft,sqm,yearbuilt
count,26.000000,124.000000,105.000000,5.070000e+02,507.000000,312.000000
mean,67.269234,3.766129,713.647644,9.475671e+04,8803.185547,1960.387817
std,26.760506,3.052678,847.602539,1.243108e+05,11548.848633,34.730198
min,3.000000,1.000000,0.000000,4.298000e+03,399.297089,1756.000000
25%,49.000000,2.000000,230.000000,3.136550e+04,2913.949097,1945.000000
50%,79.000000,2.000000,383.000000,6.666084e+04,6193.000000,1966.000000
75%,87.750000,6.000000,909.000000,1.214553e+05,11283.570312,1976.000000
max,100.000000,16.000000,4144.000000,1.675720e+06,155679.421875,2014.000000


In [4]:
buildings = buildings.copy()
buildings = buildings.drop(columns=['dataend', 'datastart', 'mainheatingtype', 'nickname', 'primaryspaceuse_abbrev', 'newweatherfilename', 'annualschedule'])

In [5]:
buildings.isna().sum()

uid                    0
energystarscore      481
heatingtype          383
industry               0
numberoffloors       383
occupants            402
primaryspaceusage      0
rating               376
sqft                   0
sqm                    0
subindustry            0
timezone               0
yearbuilt            195
dtype: int64

In [6]:
for column in buildings.columns[buildings.dtypes == 'object']:
    buildings[column] = buildings[column].fillna(value=buildings[column].value_counts().index[0])

for column in buildings.columns[buildings.dtypes == 'float32']:
    buildings[column] = buildings[column].fillna(value=buildings[column].mean())

In [7]:
buildings.isna().sum()

uid                  0
energystarscore      0
heatingtype          0
industry             0
numberoffloors       0
occupants            0
primaryspaceusage    0
rating               0
sqft                 0
sqm                  0
subindustry          0
timezone             0
yearbuilt            0
dtype: int64

In [8]:
time_series

,timestamp,consumption,uid,temperature,dew_point,humidity,sea_level_pressure,visibility,wind_direction,wind_speed,gust_speed,precipitation,events,conditions,wind_direction_degrees
0,2015-01-01 00:00:00,27.790001,Office_Abbey,2.8,-0.6,79.0,1016.000000,16.1,WNW,9.3,NaN,NaN,Calm,Mostly Cloudy,300.0
1,2015-01-01 01:00:00,27.660000,Office_Abbey,2.8,-0.6,79.0,1016.000000,16.1,WNW,9.3,NaN,NaN,Calm,Mostly Cloudy,300.0
2,2015-01-01 02:00:00,27.129999,Office_Abbey,2.8,-0.6,79.0,1016.900024,16.1,WNW,7.4,NaN,NaN,Calm,Mostly Cloudy,300.0
3,2015-01-01 03:00:00,27.340000,Office_Abbey,2.8,-1.1,76.0,1017.200012,16.1,NW,5.6,NaN,NaN,Calm,Scattered Clouds,320.0
4,2015-01-01 04:00:00,27.340000,Office_Abbey,2.2,-0.6,82.0,1017.500000,16.1,ENE,5.6,NaN,NaN,Calm,Partly Cloudy,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513811,2013-12-31 19:00:00,46.250000,UnivLab_Tracy,-2.0,-2.0,99.0,1019.000000,0.3,NNW,3.7,NaN,NaN,Fog,Heavy Fog,330.0
3513812,2013-12-31 20:00:00,46.000000,UnivLab_Tracy,-2.0,-2.0,98.0,1019.000000,NaN,North,5.6,NaN,NaN,Calm,NaN,0.0
3513813,2013-12-31 21:00:00,46.500000,UnivLab_Tracy,-3.0,-3.0,100.0,1019.000000,NaN,NW,3.7,NaN,NaN,Calm,NaN,320.0
3513814,2013-12-31 22:00:00,45.875000,UnivLab_Tracy,-3.0,-3.0,100.0,1019.000000,0.5,NNE,5.6,NaN,NaN,Fog,Heavy Fog,20.0


In [9]:
time_series = time_series.copy()
time_series = time_series.drop(columns=['precipitation'])

In [10]:
time_series.isna().sum()

timestamp                       0
consumption                     0
uid                             0
temperature                  3918
dew_point                    4218
humidity                     4346
sea_level_pressure           2235
visibility                 250245
wind_direction                  1
wind_speed                      1
gust_speed                3208490
events                          0
conditions                  62584
wind_direction_degrees        495
dtype: int64

In [11]:
time_series_fill = pd.DataFrame()
for column in time_series.columns:
    time_series_fill[column] = time_series.groupby('uid', sort=False)[column].apply(lambda x: x.ffill().bfill())

In [12]:
time_series_fill.isna().sum()

timestamp                 0
consumption               0
uid                       0
temperature               0
dew_point                 0
humidity                  0
sea_level_pressure        0
visibility                0
wind_direction            0
wind_speed                0
gust_speed                0
events                    0
conditions                0
wind_direction_degrees    0
dtype: int64

In [13]:
time_series = time_series_fill.reset_index(drop=True)

In [14]:
all_data = time_series.merge(buildings, 'inner', 'uid')

In [15]:
all_data['date'] = pd.to_datetime(all_data['timestamp']).dt.date

In [16]:
all_data['consumption_sqm'] = all_data['consumption'] / all_data['sqm']

In [17]:
all_data[['date','consumption_sqm']]

,date,consumption_sqm
0,2015-01-01,0.004032
1,2015-01-01,0.004014
2,2015-01-01,0.003937
3,2015-01-01,0.003967
4,2015-01-01,0.003967
...,...,...
3513811,2013-12-31,0.007749
3513812,2013-12-31,0.007707
3513813,2013-12-31,0.007791
3513814,2013-12-31,0.007686


In [18]:
all_data['consumption_sqm'] = all_data['consumption'] / all_data['sqm']

In [19]:
all_data[['uid', 'date', 'timestamp', 'consumption_sqm', 'sqm']]

,uid,date,timestamp,consumption_sqm,sqm
0,Office_Abbey,2015-01-01,2015-01-01 00:00:00,0.004032,6891.544434
1,Office_Abbey,2015-01-01,2015-01-01 01:00:00,0.004014,6891.544434
2,Office_Abbey,2015-01-01,2015-01-01 02:00:00,0.003937,6891.544434
3,Office_Abbey,2015-01-01,2015-01-01 03:00:00,0.003967,6891.544434
4,Office_Abbey,2015-01-01,2015-01-01 04:00:00,0.003967,6891.544434
...,...,...,...,...,...
3513811,UnivLab_Tracy,2013-12-31,2013-12-31 19:00:00,0.007749,5968.700195
3513812,UnivLab_Tracy,2013-12-31,2013-12-31 20:00:00,0.007707,5968.700195
3513813,UnivLab_Tracy,2013-12-31,2013-12-31 21:00:00,0.007791,5968.700195
3513814,UnivLab_Tracy,2013-12-31,2013-12-31 22:00:00,0.007686,5968.700195


In [20]:
grouped = all_data.groupby(['uid', 'date'], sort=False)['consumption_sqm'].sum().reset_index()

In [21]:
grouped.rename(columns={'date': 'timestamp'}, inplace=True)

In [22]:
grouped

,uid,timestamp,consumption_sqm
0,Office_Abbey,2015-01-01,0.095614
1,Office_Abbey,2015-01-02,0.223647
2,Office_Abbey,2015-01-03,0.184840
3,Office_Abbey,2015-01-04,0.162599
4,Office_Abbey,2015-01-05,0.219304
...,...,...,...
146404,UnivLab_Tracy,2013-12-27,0.175122
146405,UnivLab_Tracy,2013-12-28,0.177049
146406,UnivLab_Tracy,2013-12-29,0.184002
146407,UnivLab_Tracy,2013-12-30,0.188064


In [23]:
all_data_encoded = pd.concat([grouped], ignore_index=True)

In [24]:
all_data_encoded['consumption_sqm_log'] = np.log(all_data_encoded['consumption_sqm'])   

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
train_buildings, test_buildings = train_test_split(buildings['uid'], test_size=0.2, random_state=1010)
train_buildings, val_buildings = train_test_split(train_buildings, test_size=0.25, random_state=1010)

print('train', train_buildings.shape)
print('val', val_buildings.shape)
print('test', test_buildings.shape)

train (303,)
val (102,)
test (102,)


In [27]:
def get_data_for_buildings(buildings):
    data = all_data_encoded[all_data_encoded['uid'].isin(buildings)]
    y = data['consumption_sqm'].to_numpy().reshape(-1, 1)
    X = data.drop(columns=['timestamp', 'uid', 'consumption_sqm']).to_numpy()
    ids = data[['timestamp', 'uid']].to_numpy()
    return (X, y, ids)

X_train, y_train, ids_train = get_data_for_buildings(train_buildings)
X_val, y_val, ids_val = get_data_for_buildings(val_buildings)
X_test, y_test, ids_test = get_data_for_buildings(test_buildings)

print('train', X_train.shape, y_train.shape, ids_train.shape)
print('val', X_val.shape, y_val.shape, ids_val.shape)
print('test', X_test.shape, y_test.shape, ids_test.shape)

train (87991, 1) (87991, 1) (87991, 2)
val (29939, 1) (29939, 1) (29939, 2)
test (28479, 1) (28479, 1) (28479, 2)


In [28]:
id = 12
pd.DataFrame(X_train).to_csv(f'./data/X_train_{id}.csv', index=False, header=False)

pd.DataFrame(X_val).to_csv(f'./data/X_val_{id}.csv', index=False, header=False)
pd.DataFrame(X_test).to_csv(f'./data/X_test_{id}.csv', index=False, header=False)

pd.DataFrame(y_train).to_csv(f'./data/y_train_{id}.csv', index=False, header=False)
pd.DataFrame(y_val).to_csv(f'./data/y_val_{id}.csv', index=False, header=False)
pd.DataFrame(y_test).to_csv(f'./data/y_test_{id}.csv', index=False, header=False)

pd.DataFrame(ids_train).to_csv(f'./data/ids_train_{id}.csv', index=False, header=False)
pd.DataFrame(ids_val).to_csv(f'./data/ids_val_{id}.csv', index=False, header=False)
pd.DataFrame(ids_test).to_csv(f'./data/ids_test_{id}.csv', index=False, header=False)

pd.DataFrame(train_buildings).to_csv(f'./data/buildings_train_{id}.csv', index=False, header=False)
pd.DataFrame(val_buildings).to_csv(f'./data/buildings_val_{id}.csv', index=False, header=False)
pd.DataFrame(test_buildings).to_csv(f'./data/buildings_test_{id}.csv', index=False, header=False)